In [1]:
import pandas as pd

In [45]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Planilhas de fator de emissão

In [12]:
fator_emissao_marco = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Março.csv')
fator_emissao_abril =  pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Abril.csv')
fator_emissao_maio = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Maio.csv')
fator_emissao_junho = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Junho.csv')

In [26]:
fator_emissao_marco.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Março.csv', index = False)
fator_emissao_abril.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Abril.csv', index = False)
fator_emissao_maio.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Maio.csv', index = False)
fator_emissao_junho.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Planilhas fator de emissão atualizado/Fator_emissão_Junho.csv', index = False)

# Planilhas dos laboratórios

In [16]:
medicao_final_lab1 = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 1.csv')
medicao_final_lab2 = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 2.csv')
medicao_final_lab3 = pd.read_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 3.csv')

In [49]:
medicao_final_lab1.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 1.csv', index = False)
medicao_final_lab2.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 2.csv', index = False)
medicao_final_lab3.to_csv('D:/Backup - Dev/Artigo/Códigos TCC/Medições Finais/Medição dos computadores - Laboratório 3.csv', index = False)

# Horários de aulas e seus intervalos

In [17]:
# Armazena os horários e intervalos das aulas em dicionários
intervalos_aulas = [
    {"aula": "Aula 1", "intervalos": [((13, 0), (14, 0)), ((14, 0), (14, 40))]},
    {"aula": "Aula 2", "intervalos": [((14, 50), (15, 0)), ((15, 0), (16, 0)), ((16, 0), (16, 30))]},
    {"aula": "Aula 3", "intervalos": [((16, 40), (17, 0)), ((17, 0), (18, 0)), ((18, 0), (18, 20))]}
]

# Calcula as emissões de CO2 de cada PC, considerando o consumo de energia(kWh/minuto) e fator de emissão CO2 com base no horário

In [18]:
def calcular_emissao_por_aula(intervalos, fator_emissao, medicao_lab, dia):

    emissao = {} # Dict para armazenar resultados

    for inicio, fim in intervalos: # Percorre os intervalos fornecidos

        duracao = (fim[0] * 60 + fim[1]) - (inicio[0] * 60 + inicio[1])  # Calcula a duração dos intervalos em minutos

        hora_inicial = inicio[0] # Identifica o fator de emissão para hora inicial
        
        # Atualização: A partir das colunas 'Hora' e 'Dia' o fator de emissão é encontrado
        fator = fator_emissao.loc[(fator_emissao["Dia"] == dia) & (fator_emissao["Hora"] == hora_inicial), "Fator Emissão"].values 
        
        fator = fator[0]  # Obtém o valor do fator de emissão
        
        for _, linha in medicao_lab.iterrows(): # Percorre todas as linhas de medicao_la1(cada linha é um PC)

            consumo_por_minuto = linha["Consumo em kWh(Por minuto)"] # Coluna consumo de energia por minuto

            consumo_kwh = consumo_por_minuto * duracao # Calcula multiplicando pela duração o consumo total no intervalo

            emissao_co2 = consumo_kwh * fator # Calcula o consumo multiplicando pelo fator de emissão correspondente

            # Armazena resultados com base no ID do PC
            computador_id = int(linha['ID Computador']) # Armazena ID do PC
            if computador_id not in emissao: # Verifica se existe o PC no dict 
                emissao[computador_id] = 0
            emissao[computador_id] += emissao_co2 # Acumula emissão calculada
    return emissao # Retorna chaves: ID PC e Valores: Emissão CO2 de cada computador

# Calcula emissões de CO2 unindo os resultados das emissões por aula

In [19]:
def calcular_emissoes(intervalos_aulas, fator_emissao, medicao_lab):

    resultados = {} # Dict para armazenar resultados de emissões por dia e aula

    dias_disponiveis = fator_emissao["Dia"].unique()  # Atualização: armazena todos os dias da planilha de fator de emissão
    
    for dia in dias_disponiveis: # Atualização: Itera sobre os dias disponíveis na planilha

        resultados[dia] = {} # Atualização: Dict aninhado para armazenar os resultados por das emissões por aula para cada dia

        for aula in intervalos_aulas:
            # Atualização: Calcula as emissões para cada aula considerando o fator de emissão para o dia correspondente
            emissao = calcular_emissao_por_aula(aula["intervalos"], fator_emissao, medicao_lab, dia)
            resultados[dia][aula['aula']] = emissao # Armazena os resultados, onde Key1:dia, Key2:aula e Values:Dict{emissão}
    
    return resultados

# Exibe os resultados das emissões para cada dia, aula e PC

In [20]:
def exibir_resultados(resultados):
    print("\nResultados finais:")
    for dia, aulas in resultados.items():
        print(f"\nDia {dia}:")
        for aula, emissao in aulas.items():
            print(f"  {aula}:")
            for computador, valor in emissao.items():
                print(f"    Computador {computador}: {valor:.5f} kgCO₂")

In [25]:
fator_emissao_marco = fator_emissao_marco.rename(columns={'Março': 'Fator Emissão'})
fator_emissao_abril = fator_emissao_abril.rename(columns={'Abril': 'Fator Emissão'})
fator_emissao_maio = fator_emissao_maio.rename(columns={'Maio': 'Fator Emissão'})
fator_emissao_junho = fator_emissao_junho.rename(columns={'Junho': 'Fator Emissão'})

# Resultados das emissões de Março

In [31]:
resultados_emissoes_lab1_marco = calcular_emissoes(intervalos_aulas, fator_emissao_marco, medicao_final_lab1)
#exibir_resultados(resultados_emissoes_lab1_marco)

Laboratório 2

In [32]:
resultados_emissoes_lab2_marco = calcular_emissoes(intervalos_aulas, fator_emissao_marco, medicao_final_lab2)
#exibir_resultados(resultados_emissoes_lab2_marco)

Laboratório 3

In [33]:
resultados_emissoes_lab3_marco = calcular_emissoes(intervalos_aulas, fator_emissao_marco, medicao_final_lab3)
#exibir_resultados(resultados_emissoes_lab3_marco)

# Resultados das emissões de Abril

Laboratório 1

In [34]:
resultados_emissoes_lab1_abril = calcular_emissoes(intervalos_aulas, fator_emissao_abril, medicao_final_lab1)
#exibir_resultados(resultados_emissoes_lab1_abril)

Laboratório 2

In [35]:
resultados_emissoes_lab2_abril = calcular_emissoes(intervalos_aulas, fator_emissao_abril, medicao_final_lab2)
#exibir_resultados(resultados_emissoes_lab2_abril)

Laboratório 3

In [36]:
resultados_emissoes_lab3_abril = calcular_emissoes(intervalos_aulas, fator_emissao_abril, medicao_final_lab3)
#exibir_resultados(resultados_emissoes_lab3_abril)

# Resultados emissões de Maio

Laboratório 1

In [37]:
resultados_emissoes_lab1_maio = calcular_emissoes(intervalos_aulas, fator_emissao_maio, medicao_final_lab1)
#exibir_resultados(resultados_emissoes_lab1_maio)

Laboratório 2

In [38]:
resultados_emissoes_lab2_maio = calcular_emissoes(intervalos_aulas, fator_emissao_maio, medicao_final_lab2)
#exibir_resultados(resultados_emissoes_lab2_maio)

Laboratório 3

In [39]:
resultados_emissoes_lab3_maio = calcular_emissoes(intervalos_aulas, fator_emissao_maio, medicao_final_lab3)
#exibir_resultados(resultados_emissoes_lab3_maio)

# Resultados das emissões de junho

Laboratório 1

In [40]:
resultados_emissoes_lab1_junho = calcular_emissoes(intervalos_aulas, fator_emissao_junho, medicao_final_lab1)
#exibir_resultados(resultados_emissoes_lab1_junho)

Laboratório 2

In [41]:
resultados_emissoes_lab2_junho = calcular_emissoes(intervalos_aulas, fator_emissao_junho, medicao_final_lab2)
#exibir_resultados(resultados_emissoes_lab2_junho)

Laboratório 3

In [42]:
resultados_emissoes_lab3_junho = calcular_emissoes(intervalos_aulas, fator_emissao_junho, medicao_final_lab3)
#exibir_resultados(resultados_emissoes_lab3_junho)

# Cria planilha nova com todos os resultados

In [43]:
def cria_planilha_csv(resultado_labx, nome_arquivo):
    
    dados = [] # Armazena os dados para transformação em DataFrame
    
    for dia, aulas in resultado_labx.items(): # Itera sobre dias, aulas e PC no dict
        for aula, computadores in aulas.items():
            for computador, valor in computadores.items():

                dados.append([aula, f"{int(computador)}", f"{dia}", valor]) # Adiciona os dados de cada PC na lista
    
    nova_planilha = pd.DataFrame(dados, columns = ["Horario Aula", "PC", "Data", "Emissao (kgCO₂)"]) # Cria DataFrame a partir da lista com colunas nomeadas
    
    nova_planilha["PC"] = nova_planilha["PC"].astype(int) # Converte a coluna 'PC' para inteiro garantindo ordenação
    nova_planilha["Data"] = nova_planilha["Data"].astype(int) # Converte a coluna 'Data' para inteiro garantindo ordenação
    nova_planilha = nova_planilha.sort_values(by = ["Horario Aula", "PC"]) # Ordena o DataFrame pelas colunas 'Horario Aula' e 'PC'
    
    nova_planilha["Emissao (kgCO₂)"] = nova_planilha["Emissao (kgCO₂)"].astype(float).round(5) # Arredonda os valores para 5 casas decimais
    
    nova_planilha_pivotada = nova_planilha.pivot(index=["Horario Aula", "PC"], columns = "Data", values = "Emissao (kgCO₂)") # Cria uma tabela pivotada organizando os dados

    nova_planilha_pivotada.to_csv(nome_arquivo, sep = ',', encoding = 'utf-8') # Salva a tabela criada


# Planilhas mês de Março

In [44]:
cria_planilha_csv(resultados_emissoes_lab1_marco, "Resultado_marco_lab1.csv")

In [45]:
cria_planilha_csv(resultados_emissoes_lab2_marco, "Resultado_marco_lab2.csv")

In [46]:
cria_planilha_csv(resultados_emissoes_lab3_marco, "Resultado_marco_lab3.csv")

# Planilhas mês de Abril

In [47]:
cria_planilha_csv(resultados_emissoes_lab1_abril, "Resultado_abril_lab1.csv")

In [48]:
cria_planilha_csv(resultados_emissoes_lab2_abril, "Resultado_abril_lab2.csv")

In [49]:
cria_planilha_csv(resultados_emissoes_lab3_abril, "Resultado_abril_lab3.csv")

# Planilhas mês de Maio

In [50]:
cria_planilha_csv(resultados_emissoes_lab1_maio, "Resultado_maio_lab1.csv")

In [51]:
cria_planilha_csv(resultados_emissoes_lab2_maio, "Resultado_maio_lab2.csv")

In [52]:
cria_planilha_csv(resultados_emissoes_lab3_maio, "Resultado_maio_lab3.csv")

# Planilhas mês de Junho

In [53]:
cria_planilha_csv(resultados_emissoes_lab1_junho, "Resultado_junho_lab1.csv")

In [54]:
cria_planilha_csv(resultados_emissoes_lab2_junho, "Resultado_junho_lab2.csv")

In [55]:
cria_planilha_csv(resultados_emissoes_lab3_junho, "Resultado_junho_lab3.csv")